In [3]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import math
import pandas as pd

### web-scraping code

In [162]:
# "apartments" or 'houses'

properties_to_look_for = "apartments"

In [163]:
def url_selection(properties_to_look_for):

    if properties_to_look_for == 'apartments':
        url = 'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
#         url_several = f'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

    elif properties_to_look_for == 'houses':
        url = 'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
#         url_several = f'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

    return (url)

# url_selection(properties_to_look_for)

In [166]:
# MULTIPLE PAGES:

url = url_selection(properties_to_look_for)
    
count_number = 0

permalink_list = []
titles_list = []
price_list = []
bedrooms_number_list = []
full_bathrooms_list = []
thumbnail_list = []
announcement_id_list = []
address_street_list = []
colonia_list = []
city_list = []
state_list = []
property_type_list = []
type_of_operation_list = []
currency_list = []
real_state_agency_list = []

properties_feature_list = []  
        

executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

browser.visit(url)
browser.is_element_not_present_by_id('list-card-container', wait_time=2)
html = browser.html
soup = bs(html, "html.parser")

number_of_results = soup.find("h1", class_ = "list-result-title").text.split(" ")[0].replace(',','')
number_of_results = int(number_of_results)

total_paging_times = math.ceil(number_of_results/24) + 1
print(F'properties_to_look_for: {properties_to_look_for}, total_paging_times: {total_paging_times}')
print('')


for i in range(1,total_paging_times):   

    
#   HERE IS WHERE I PUT THE URL
    if i == 1:
        url = url
    else:
        if properties_to_look_for == 'apartments':
            url = f'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

        elif properties_to_look_for == 'houses':
            url = f'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-pagina-{i}-q-ciudad-de-mexico.html'

    
    
    print(f'URL VISITED: {url}')
    print('')
    
    browser.visit(url)
    browser.is_element_not_present_by_id('list-card-container', wait_time=2)
    html = browser.html
    soup = bs(html, "html.parser")
    
    publications_list = soup.find_all("div", class_ = "postingCard")

    for x in range(len(publications_list)):

        count_number += 1

    #     PROPERTY_TYPE:

        if 'departamento' in url:
            property_type = 'Departamento'
        elif 'casa' in url:
            property_type = 'Casa'

        property_type_list.append(property_type)

    #     TYPE OF OPERATION:

        if 'venta' in url:
            type_of_operation = 'Venta'
        elif 'renta' in url:
            type_of_operation = 'Renta'

        type_of_operation_list.append(type_of_operation)

    #   PROPERTY_ID
        property_id = publications_list[x]["data-id"]
        announcement_id_list.append(property_id)


    #     HYPERLINK    
        hyperlink = f'https://www.inmuebles24.com{publications_list[x]["data-to-posting"]}'
        permalink_list.append(hyperlink)

    #     ------SUB_SOUP-----

        sub_soup = publications_list[x]

    #     PROPERTY_TITLE
        ad_title = sub_soup.find_all("h2", class_ = "postingCardTitle")[0].get_text(strip=True)
        titles_list.append(ad_title)

    #     PROPERTY PRICE
        property_price = sub_soup.find_all("span", class_ = "firstPrice")[0].get_text(strip=True).replace('MN ', '').replace(',', '')
        price_list.append(property_price)

    #     CURRENCY:

        if 'USD' in property_price:
            currency = 'USD'
        else:
            currency = 'MXN'

        currency_list.append(currency)

    # #     print(f'id: {property_id} , title: {ad_title} , {hyperlink} , price: {property_price}')

    #     THUMBNAIL:

        try:
            thumbnail = sub_soup.find_all("img")[0]['data-src'].strip()
            thumbnail_list.append(thumbnail)
        except:
            thumbnail = sub_soup.find_all("img")[0]['src'].strip()
            thumbnail_list.append(thumbnail)


        #   LOCATION TITLE

        street = sub_soup.find_all("span", class_ = "postingCardLocationTitle")[0].get_text(strip=True)

        address = sub_soup.find_all("span", class_ = "postingCardLocation")[0].get_text(strip=True).split(",")
        colonia = address[0]
        city = address[1]
        state = 'Distrito Federal'

        address_street_list.append(street)
        colonia_list.append(colonia)
        city_list.append(city)
        state_list.append(state)

    #     REAL ESTATE AGENCY:
        
        try:
            real_state_agency = sub_soup.find_all("div", class_ = "publisherLogo")[0].a.img['src'].split("/")[-1]

            if real_state_agency == 'no-logo.svg':
                real_state_agency = 'trato_directo'
            else:
                real_state_agency = real_state_agency.replace('.jpg', '').replace('logo_', '')
        
        except:
            real_state_agency = 'not_available'

        real_state_agency_list.append(real_state_agency)

    #     FEATURES LIST:

        inside_list = []

        features_list = sub_soup.find_all("ul", class_ = "postingCardMainFeatures")[0].find_all("li")

        for j in range(len(features_list)):
            feature = features_list[j].get_text(strip = True).replace("\n", " ").replace("\t", "")
            inside_list.append(feature)

    #         BEDROOM NUMBER:

            if 'rec.' in feature:
                bedrooms_number = feature[0:2]
                bedrooms_number_list.append(bedrooms_number)
                
    #         BATHROOMS NUMBER:

            if 'baños' in feature or 'baño' in feature:
                bathrooms_number = feature[0:2]
                full_bathrooms_list.append(bathrooms_number)
                
        if len(bedrooms_number_list) != count_number:
            bedrooms_number_list.append("not_available")

        if len(full_bathrooms_list) != count_number:
            full_bathrooms_list.append("not_available")

        properties_feature_list.append(inside_list)

        print(f"""ID: {count_number} - property_id: {property_id},
        hyperlink: {hyperlink}
        """)
        
    time.sleep(2.5)

    #   street: {street}, colonia: {colonia}, city: {city}, state: {state},      
#     ad_title: {ad_title}, property_price: {property_price},
    #     thumbnail: {thumbnail} bedrooms_number: {bedrooms_number}, bathrooms_number: {bathrooms_number},
    #     real_state_agency: {real_state_agency}

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Intel NUC\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


 
properties_to_look_for: apartments, total_paging_times: 77

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html

ID: 1 - property_id: 52913725,
        hyperlink: https://www.inmuebles24.com/propiedades/bosque-6060-52913725.html
        
ID: 2 - property_id: 54456938,
        hyperlink: https://www.inmuebles24.com/propiedades/sky-tower-lindavista-54456938.html
        
ID: 3 - property_id: 55534814,
        hyperlink: https://www.inmuebles24.com/propiedades/ultimo-departamento-departamento-en-venta-tlalpan-55534814.html
        
ID: 4 - property_id: 60679141,
        hyperlink: https://www.inmuebles24.com/propiedades/chicle-160-60679141.html
        
ID: 5 - property_id: 56515561,
        hyperlink: https://www.inmuebles24.com/propiedades/up-santa-fe-56515561.html
        
ID: 6 - property_id: 60859941,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-penn

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-4-q-ciudad-de-mexico.html

ID: 73 - property_id: 60275707,
        hyperlink: https://www.inmuebles24.com/propiedades/increible-departamento-en-rio-sena-49-a-una-cuadra-60275707.html
        
ID: 74 - property_id: 59690152,
        hyperlink: https://www.inmuebles24.com/propiedades/estrena-uno-de-7-departamentos-exclusivos-de-lujo-59690152.html
        
ID: 75 - property_id: 60111791,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-100-remodelado-muy-cerca-de-narvarte-60111791.html
        
ID: 76 - property_id: 60006686,
        hyperlink: https://www.inmuebles24.com/propiedades/comodo-departamento-en-av-cuauhtemoc-1176-col.-letran-60006686.html
        
ID: 77 - property_id: 59891510,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-guadalupe-inn-59891510.html
        
ID: 78 - property_id

ID: 121 - property_id: 60872202,
        hyperlink: https://www.inmuebles24.com/propiedades/venta-departamento-en-iztapalapa-ciudad-de-mexico-v-60872202.html
        
ID: 122 - property_id: 60779136,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-zona-sur-de-ciudad-de-mexico-60779136.html
        
ID: 123 - property_id: 60806834,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-a-la-venta-en-san-angel-ciudad-de-mexico-60806834.html
        
ID: 124 - property_id: 59567613,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-escandon-miguel-hidalgo-59567613.html
        
ID: 125 - property_id: 59510689,
        hyperlink: https://www.inmuebles24.com/propiedades/hermoso-y-lujoso-ph-en-el-corazon-de-la-ciudad-de-59510689.html
        
ID: 126 - property_id: 57010240,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-ubicado-en-ciudad-de-mexico-57010240.html
        
ID: 

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-9-q-ciudad-de-mexico.html

ID: 193 - property_id: 59282629,
        hyperlink: https://www.inmuebles24.com/propiedades/tetelpan-departamento-venta-desierto-de-los-leones-a-59282629.html
        
ID: 194 - property_id: 60163339,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-1-recamara-1-bano.-puerta-alameda-centro-60163339.html
        
ID: 195 - property_id: 54919891,
        hyperlink: https://www.inmuebles24.com/propiedades/exclusivo-departamento-de-125-m-sup2--2-54919891.html
        
ID: 196 - property_id: 59410080,
        hyperlink: https://www.inmuebles24.com/propiedades/romasur-a-estrenar-2recamaras-2banos-1auto1bodega-59410080.html
        
ID: 197 - property_id: 59292759,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-paseo-de-la-reforma-59292759.html
        
ID: 198 - propert

...

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-64-q-ciudad-de-mexico.html

ID: 1513 - property_id: 60539376,
        hyperlink: https://www.inmuebles24.com/propiedades/hermoso-ph-en-venta-en-del-valle-60539376.html
        
ID: 1514 - property_id: 59039093,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-remodelado-en-venta-en-hipodromo-59039093.html
        
ID: 1515 - property_id: 60529070,
        hyperlink: https://www.inmuebles24.com/propiedades/comodo-departamento-interior.-pb-3r-2b-1e-60529070.html
        
ID: 1516 - property_id: 60528128,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-roma-norte-60528128.html
        
ID: 1517 - property_id: 60516441,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-industrial-60516441.html
        
ID: 1518 - property_id: 60511816,
        hyperlink: https:/

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-67-q-ciudad-de-mexico.html

ID: 1585 - property_id: 60582330,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-mujeres-ilustres-60582330.html
        
ID: 1586 - property_id: 60579919,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-transvaal-126-eje-2-norte-60579919.html
        
ID: 1587 - property_id: 60575057,
        hyperlink: https://www.inmuebles24.com/propiedades/hermoso-y-amplio-departamento-planta-baja-en-venta-en-60575057.html
        
ID: 1588 - property_id: 60574853,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-fuentes-del-pedregal-60574853.html
        
ID: 1589 - property_id: 60573354,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-insurgentes-cuicuilco-60573354.html
        
ID: 1590 - pr

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-70-q-ciudad-de-mexico.html

ID: 1657 - property_id: 60353227,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-industrial-vallejo-60353227.html
        
ID: 1658 - property_id: 60352900,
        hyperlink: https://www.inmuebles24.com/propiedades/bonito-departamento-remodelado-en-venta-en-merced-60352900.html
        
ID: 1659 - property_id: 60352750,
        hyperlink: https://www.inmuebles24.com/propiedades/depto-remodelado-78-m-sup2-60352750.html
        
ID: 1660 - property_id: 59674298,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-la-narvarte-59674298.html
        
ID: 1661 - property_id: 60347464,
        hyperlink: https://www.inmuebles24.com/propiedades/casas-grandes-130-60347464.html
        
ID: 1662 - property_id: 60345220,
        hyperlink: https://www.inmuebles24.

URL VISITED: https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-pagina-73-q-ciudad-de-mexico.html

ID: 1729 - property_id: 60497017,
        hyperlink: https://www.inmuebles24.com/propiedades/el-unico-desarrollo-en-la-roma-con-casa-club-60497017.html
        
ID: 1730 - property_id: 60496457,
        hyperlink: https://www.inmuebles24.com/propiedades/excelentes-deptos-con-vista-al-parque-espana-y-reforma-60496457.html
        
ID: 1731 - property_id: 60496161,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-ampliacion-granada-60496161.html
        
ID: 1732 - property_id: 60494140,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-lindavista-60494140.html
        
ID: 1733 - property_id: 60493507,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-unidad-infonavit-independencia-60493507.html
        
ID: 1734 - property_id:

ID: 1777 - property_id: 60315907,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-en-la-colonia-san-miguel-60315907.html
        
ID: 1778 - property_id: 60313451,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-exclusivo-en-venta-en-lindavista-60313451.html
        
ID: 1779 - property_id: 60308122,
        hyperlink: https://www.inmuebles24.com/propiedades/pre-venta-ubicacion-privilegiada-alta-plusvalia-60308122.html
        
ID: 1780 - property_id: 60369188,
        hyperlink: https://www.inmuebles24.com/propiedades/departamento-en-venta-$6-350-000-mxn-cuauhtemoc-60369188.html
        
ID: 1781 - property_id: 60988091,
        hyperlink: https://www.inmuebles24.com/propiedades/venta-departamento-60988091.html
        
ID: 1782 - property_id: 60990830,
        hyperlink: https://www.inmuebles24.com/propiedades/incrementa-el-valor-de-tu-dinero-invierte-en-el-60990830.html
        
ID: 1783 - property_id: 60990608,
        hyperl

In [169]:
properties_feature_list

[['18 deptos.', '3 rec.', '12 a 66   m²'],
 ['184 deptos.', '2 rec.', '75 a 169   m²'],
 ['7 deptos.', '3 rec.', '335   m²'],
 ['40 deptos.', '2 rec.', '56 a 57   m²'],
 ['177 deptos.', '1 a 2 rec.', '55 a 84   m²'],
 ['123 m²', '4 rec.', '2  baños', '1  estac.'],
 ['102 m²', '3 rec.', '2  baños', '1  estac.'],
 ['82 m²', '3 rec.', '2  baños', '2  estac.'],
 ['155 m²', '3 rec.', '2  baños', '2  estac.'],
 ['290 m²', '3 rec.', '3  baños', '3  estac.'],
 ['103 m²', '3 rec.', '2  baños', '1  estac.'],
 ['328 m²', '3 rec.', '3  baños', '4  estac.'],
 ['87 m²', '2 rec.', '2  baños', '2  estac.'],
 ['41 m²', '2 rec.', '1  baño'],
 ['25 m²', '2 rec.', '2  baños', '2  estac.'],
 ['136 m²', '2 rec.', '2  baños', '1  estac.'],
 ['63 m²', '2 rec.', '2  baños', '1  estac.'],
 ['192 m²', '3 rec.', '2  baños', '3  estac.'],
 ['101 m²', '2 rec.', '3  baños', '2  estac.'],
 ['92 m²', '3 rec.', '2  baños', '2  estac.'],
 ['55 m²', '2 rec.', '1  baño', '1  estac.'],
 ['901 m²', '3 rec.', '2  baños', '2 

In [167]:
properties_24_df = pd.DataFrame({
    'ad_id': announcement_id_list,
    'title': titles_list,
    'type_of_operation': type_of_operation_list,
    'property_type': property_type_list,
    'price': price_list,
    "currency" : currency_list,
    'bedrooms_number': bedrooms_number_list,
    'full_bathrooms_number': full_bathrooms_list,
    'real_state_agency': real_state_agency_list,
    'permalink': permalink_list,
    'thumbnail': thumbnail_list,
    'address_street': address_street_list,
    'neighborhood': colonia_list,
    'city': city_list,
    'state': state_list,
})

properties_24_df.head()

ValueError: arrays must all be same length

In [139]:
properties_24_df.shape

(48, 15)

In [140]:
properties_24_df.title

0                                           Bosque 6060
1                                  Sky Tower Lindavista
2     Último Departamento, Departamento en Venta Tla...
3                                            Chicle 160
4                                           Up Santa Fe
5         Departamento en Pennsylvania Ciudad de México
6             Departamento en Progreso Ciudad de México
7               Departamento en Tanana Ciudad de México
8     Venta de Departamento en Polanco, Ciudad de Mé...
9      Se Vende Departamento en Lomas, Ciudad de México
10    Departamento en Venta en La Colonia del Valle,...
11    Departamento en Venta en Una de Las Mejores Co...
12    Departamentos en Venta en Calle Amores, Ciudad...
13    Departamento en Venta, en Benito Juárez Ciudad...
14    Departamento en Venta Guadalupe, Tlalpan, Ciud...
15    Departamento en Venta, Masaryk Polanco, Ciudad...
16    Departamento Nuevo en Venta Al Sur en La Ciuda...
17    Vendo Departamento Colonia Polanco Ciudad 

In [141]:
properties_24_df[properties_24_df.duplicated(subset=['ad_id'])]


,ad_id,title,type_of_operation,property_type,price,currency,bedrooms_number,full_bathrooms_number,real_state_agency,permalink,thumbnail,address_street,neighborhood,city,state


In [170]:
browser.quit()

### DEPARTAMENTO QUERY

In [23]:
# DEPARTAMENTO QUERY:

url = 'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
url

'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'

### Debugger code

In [168]:
fields = [announcement_id_list,
    titles_list,
    type_of_operation_list,
    property_type_list,
    price_list,
    currency_list,
    bedrooms_number_list,
    full_bathrooms_list,
    real_state_agency_list,
    permalink_list,
    thumbnail_list,
    address_street_list,
    colonia_list,
    city_list,
    state_list]


for x in range(len(fields)):
    print(f'field {x+1}, length: {len(fields[x])}')

field 1, length: 1816
field 2, length: 1816
field 3, length: 1816
field 4, length: 1816
field 5, length: 1816
field 6, length: 1816
field 7, length: 1803
field 8, length: 1816
field 9, length: 1816
field 10, length: 1816
field 11, length: 1816
field 12, length: 1816
field 13, length: 1816
field 14, length: 1816
field 15, length: 1816


### Analysis

In [92]:
properties_24_df['price'].unique()

array(['11312978', '3210000', '7950000', '2052000', '3701095', '4480000',
       '3650000', '13260000', '11290000', '4300000', '12499000',
       '4844237', '1417500', '4280000', '7200000', '2390000', '15500000',
       '4900000', '4400000', '2598989', '5500000', '2980000'],
      dtype=object)

In [15]:
properties_feature_list

[['18 deptos.', '3 rec.', '12 a 66   m²'],
 ['184 deptos.', '2 rec.', '75 a 169   m²'],
 ['7 deptos.', '3 rec.', '335   m²'],
 ['40 deptos.', '2 rec.', '56 a 57   m²'],
 ['177 deptos.', '1 a 2 rec.', '55 a 84   m²'],
 ['123 m²', '4 rec.', '2  baños', '1  estac.'],
 ['102 m²', '3 rec.', '2  baños', '1  estac.'],
 ['82 m²', '3 rec.', '2  baños', '2  estac.'],
 ['155 m²', '3 rec.', '2  baños', '2  estac.'],
 ['290 m²', '3 rec.', '3  baños', '3  estac.'],
 ['103 m²', '3 rec.', '2  baños', '1  estac.'],
 ['328 m²', '3 rec.', '3  baños', '4  estac.'],
 ['87 m²', '2 rec.', '2  baños', '2  estac.'],
 ['41 m²', '2 rec.', '1  baño'],
 ['25 m²', '2 rec.', '2  baños', '2  estac.'],
 ['136 m²', '2 rec.', '2  baños', '1  estac.'],
 ['63 m²', '2 rec.', '2  baños', '1  estac.'],
 ['192 m²', '3 rec.', '2  baños', '3  estac.'],
 ['92 m²', '3 rec.', '2  baños', '2  estac.'],
 ['55 m²', '2 rec.', '1  baño', '1  estac.'],
 ['901 m²', '3 rec.', '2  baños', '2  estac.'],
 ['80 m²', '2 rec.', '2  baños', '1  

In [16]:
properties_feature_list

[['18 deptos.', '3 rec.', '12 a 66   m²'],
 ['184 deptos.', '2 rec.', '75 a 169   m²'],
 ['7 deptos.', '3 rec.', '335   m²'],
 ['40 deptos.', '2 rec.', '56 a 57   m²'],
 ['177 deptos.', '1 a 2 rec.', '55 a 84   m²'],
 ['123 m²', '4 rec.', '2  baños', '1  estac.'],
 ['102 m²', '3 rec.', '2  baños', '1  estac.'],
 ['82 m²', '3 rec.', '2  baños', '2  estac.'],
 ['155 m²', '3 rec.', '2  baños', '2  estac.'],
 ['290 m²', '3 rec.', '3  baños', '3  estac.'],
 ['103 m²', '3 rec.', '2  baños', '1  estac.'],
 ['328 m²', '3 rec.', '3  baños', '4  estac.'],
 ['87 m²', '2 rec.', '2  baños', '2  estac.'],
 ['41 m²', '2 rec.', '1  baño'],
 ['25 m²', '2 rec.', '2  baños', '2  estac.'],
 ['136 m²', '2 rec.', '2  baños', '1  estac.'],
 ['63 m²', '2 rec.', '2  baños', '1  estac.'],
 ['192 m²', '3 rec.', '2  baños', '3  estac.'],
 ['92 m²', '3 rec.', '2  baños', '2  estac.'],
 ['55 m²', '2 rec.', '1  baño', '1  estac.'],
 ['901 m²', '3 rec.', '2  baños', '2  estac.'],
 ['80 m²', '2 rec.', '2  baños', '1  

In [38]:
properties_24_df.shape

(24, 7)

In [12]:
features_list = sub_soup.find_all("ul", class_ = "postingCardMainFeatures")[0].find_all("li")

additional_list = []

for x in range(len(features_list)):
    test = features_list[x].get_text(strip = True).replace("\n", " ").replace("\t", "")
    additional_list.append(test)
#     print(test)

additional_list

['18 deptos.', '3 rec.', '12 a 66   m²']

## Scrape of each webpage

In [221]:
# THIS SHOULD BE OBTAINED FROM EACH PAGES' WEBSITE:

square_meters_list = []
unit_list = []

item_condition = []

if 'm²' in feature:
    square_meters_list.append(feature.replace("m²", ""))
    unit_list.append('m²') 

In [14]:
import time

image_details_coordinates_list = []
latitude_list = []
longitude_list = []

count_number = 0

for x in range(3):
    
    count_number += 1

    url = permalink_list[x]

    browser.visit(url)

    time.sleep(1)

    html = browser.html

    soup = bs(html, "html.parser")
    
#     SCRAPE OF IMAGE COORDINATES:

    map_image = soup.find('div', class_ = 'article-map')

    image_details = map_image.img['src'].split("&")

    print(permalink_list[x])
    
    for x in range(len(image_details)):
        if 'markers' in image_details[x]:
            image_details_coordinates = image_details[x].replace('markers=', '').split(",")
            image_details_coordinates_list.append(image_details_coordinates)
            
            latitude = image_details_coordinates[0]
            longitude = image_details_coordinates[1]            
            latitude_list.append(latitude)
            longitude_list.append(longitude)
                        
    if len(image_details_coordinates_list) != count_number:
        for x in range(len(image_details)):
            if 'center' in image_details[x]:
                image_details_coordinates = image_details[x].replace('center=', '').split(",")
                image_details_coordinates_list.append(image_details_coordinates)

                latitude = image_details_coordinates[0].split("?")[1]
                longitude = image_details_coordinates[1]            
                latitude_list.append(latitude)
                longitude_list.append(longitude)

    print(latitude, longitude)

https://www.inmuebles24.com/propiedades/bosque-6060-52913725.html
19.331155400000000 -99.256939199999980
https://www.inmuebles24.com/propiedades/sky-tower-lindavista-54456938.html
19.481846100000000 -99.129833500000020
https://www.inmuebles24.com/propiedades/ultimo-departamento-departamento-en-venta-tlalpan-55534814.html
19.279131400000000 -99.140570900000000


In [261]:
pd.DataFrame({
    "bedrooms_number" : bedrooms_number_list
})

,bedrooms_number
0,3
1,2
2,3
3,2
4,1
5,2
6,4
7,3
8,3
9,2


In [126]:
browser.quit()